## Zonal Costs

In [112]:
# Third-party packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Local imports
from utils import get_data
from utils import tech_order, tech_colors
from utils import get_data_sftp

from datetime import datetime
from pytz import timezone
import os

from matplotlib import cm
from matplotlib.colors import Normalize
from mpl_toolkits.mplot3d import Axes3D

pd.options.display.float_format = '{:,.2f}'.format

### Scenarios

In [113]:
'''
scenario = ['baseline_pverde_islanded_ldes_10', 
            'baseline_pverde_islanded_ldes_15']
short_names={'baseline_pverde_islanded_ldes_5':5, 
            'baseline_pverde_islanded_ldes_10':10, 
            'baseline_pverde_islanded_ldes_15':15}

order={5:0, 
        10:1, 
        15:2}

folder_to_save_results='results_base_islanded_storage_sensitivity/'


scenario = ['baseline_pverde_ldes_5', 
            'baseline_pverde_ldes_10']
short_names={'baseline_pverde_ldes_5':5, 
            'baseline_pverde_ldes_10':10}
order={5:0, 
        10:1}
folder_to_save_results='results_base_storage_sensitivity/'
'''

scenario = ['scenario_176']
short_names={'scenario_176':1}
order={1:0}

folder_to_save_results='results_base_storage_sensitivity/'
#Check if the directory exists. If not, then create the directory.
if not os.path.exists(folder_to_save_results):
    os.makedirs(folder_to_save_results)

In [114]:
#zones_under_analysis
analysis_zones=[] #these are coastal load zones

In [115]:
#Functions

def default_value(Dataframe, column, default_value):
    if column not in Dataframe.columns:
        Dataframe[column] = default_value
    else:
        Dataframe[column] = Dataframe[column].replace('.',0).astype(float)

    return Dataframe

def crf(ir, t):
    if ir==0:
        return 1/t
    else:
        return (ir/(1-(1+ir)**(-t)))

def uniform_series_to_present_value(dr, t):
    if dr==0:
        return t
    else:
        return ((1-(1+dr)**(-t))/dr)

def future_to_present_value(dr, t):
    return (1+dr)**(-t)

def present_to_future_value(ir, t):
    return (1+ir)**(t)

def gen_build_can_operate_in_period(gen_max_age, build_year, investment_period, period_start, period_length_years):
        if build_year==investment_period:
            online = period_start
        else:
            online = build_year
        
        retirement = online + gen_max_age

        if online <= period_start + 0.5*period_length_years < retirement :
            return 1
        else:
            return 0


### Time-related calculations

In [116]:
hours_per_year = 8766

fname="timepoints.csv"
timepoints = get_data(scenario, fname, fpath='inputs')
timepoints.columns= timepoints .columns.str.lower()
timepoints.rename(columns={'timepoint_id': 'timepoint'}, inplace=True)
timepoints = timepoints .replace({"scenario": short_names})

fname="timeseries.csv"
timeseries = get_data(scenario, fname, fpath='inputs')
timeseries.columns= timeseries .columns.str.lower()
timeseries = timeseries .replace({"scenario": short_names})

fname="periods.csv"
periods = get_data(scenario, fname, fpath='inputs')
periods.columns= periods .columns.str.lower()
periods.rename(columns={'investment_period': 'ts_period'}, inplace=True)
periods = periods .replace({"scenario": short_names})

#Merge loads with timepoints
time_info=pd.merge(left=timeseries,right=timepoints, on=['timeseries', 'scenario'])
time_info=pd.merge(left=time_info,right=periods , on=['scenario', 'ts_period'])

time_info['tp_weight']=time_info['ts_duration_of_tp']*time_info['ts_scale_to_period']

period_info=time_info.pivot_table(index=['scenario','ts_period'], values='tp_weight',aggfunc=np.sum )
period_info.reset_index(inplace=True)
period_info.rename(columns={'tp_weight': 'hours_in_period'}, inplace=True)
period_info = pd.merge(left=period_info, right = periods, on=['scenario', 'ts_period'])
period_info['err_plain'] = (period_info['period_end'] - period_info['period_start'])*hours_per_year - period_info['hours_in_period']
period_info['err_add_one'] =  (period_info['period_end'] + 1 - period_info['period_start'])*hours_per_year - period_info['hours_in_period']
period_info.loc[:, 'add_one_to_period_end_rule']= period_info.apply(lambda x: 1 if np.absolute(x['err_add_one'])<np.absolute(x['err_plain']) else 0, axis=1)
period_info['period_length_years'] = period_info['period_end'] - period_info['period_start'] + period_info['add_one_to_period_end_rule']


fname = "financials.csv"
financials = get_data(scenario, fname, fpath='inputs')
financials  = financials  .replace({"scenario": short_names})

period_info = pd.merge(left=period_info, right=financials, on='scenario')
period_info.loc[:, 'bring_annual_costs_to_base_year']=period_info.apply(lambda x: uniform_series_to_present_value(x['discount_rate'], x['period_length_years'])
                                                                        *future_to_present_value(x['discount_rate'], x['period_start'] - x['base_financial_year']), axis=1)
period_info.rename(columns={'ts_period': 'investment_period'}, inplace=True)


exception
timestamp in column


### Generation Fixed Costs

In [117]:
generation_projects_info = get_data(scenario, "generation_projects_info.csv" , fpath='inputs')
generation_projects_info  = generation_projects_info.replace({"scenario": short_names})
generation_projects_info  = default_value(generation_projects_info, "gen_connect_cost_per_mw", 0)

gen_build_costs = get_data(scenario, "gen_build_costs.csv", fpath='inputs')
gen_build_costs  = gen_build_costs  .replace({"scenario": short_names})
gen_build_costs  = default_value(gen_build_costs, "gen_overnight_cost", 0)
gen_build_costs  = default_value(gen_build_costs, "gen_fixed_om", 0)
gen_build_costs  = default_value(gen_build_costs, "gen_storage_energy_overnight_cost", 0)

gen_build_costs_extended = pd.merge(left=gen_build_costs , right=financials, on='scenario')
gen_build_costs_extended = pd.merge(left=gen_build_costs_extended , right=generation_projects_info[['GENERATION_PROJECT', 'gen_load_zone', "gen_connect_cost_per_mw", 'gen_max_age', 'scenario']], on=['GENERATION_PROJECT','scenario'])
gen_build_costs_extended = gen_build_costs_extended[['GENERATION_PROJECT', 'gen_load_zone', 'build_year', 'gen_overnight_cost', 'gen_fixed_om', 'gen_storage_energy_overnight_cost', 'gen_max_age', "gen_connect_cost_per_mw", "interest_rate", "scenario"]]

gen_build_costs_extended.loc[:,'gen_capital_cost_annual'] = gen_build_costs_extended.apply(lambda x: (x['gen_overnight_cost'] + x['gen_connect_cost_per_mw']) * crf(x['interest_rate'], x['gen_max_age']), axis=1)
gen_build_costs_extended.loc[:,'storage_energy_capital_cost_annual'] = gen_build_costs_extended.apply(lambda x: x['gen_storage_energy_overnight_cost'] * crf(x['interest_rate'], x['gen_max_age']), axis=1)

BuildGen = get_data(scenario, "BuildGen.csv")
BuildGen  = BuildGen.replace({"scenario": short_names})
BuildGen.rename(columns={'GEN_BLD_YRS_1':'GENERATION_PROJECT', "GEN_BLD_YRS_2": "build_year"},inplace=True)

gen_build_costs_extended = pd.merge(left = gen_build_costs_extended, right=BuildGen, on=['GENERATION_PROJECT', 'build_year', 'scenario'])
gen_build_costs_extended

,GENERATION_PROJECT,gen_load_zone,build_year,gen_overnight_cost,gen_fixed_om,gen_storage_energy_overnight_cost,gen_max_age,gen_connect_cost_per_mw,interest_rate,scenario,gen_capital_cost_annual,storage_energy_capital_cost_annual,BuildGen
0,77333,CA_SCE_CEN,2020,"1,393,459.89","47,047.10",0.00,20,"87,089.10",0.05,1,"118,803.08",0.00,0.00
1,77333,CA_SCE_CEN,2030,"1,247,479.88","38,866.50",0.00,20,"87,089.10",0.05,1,"107,089.27",0.00,0.00
2,77333,CA_SCE_CEN,2040,"1,120,977.80","35,882.70",0.00,20,"87,089.10",0.05,1,"96,938.41",0.00,43.00
3,77333,CA_SCE_CEN,2050,"1,042,433.46","33,692.00",0.00,20,"87,089.10",0.05,1,"90,635.81",0.00,0.00
4,77334,CA_SCE_CEN,2020,"1,393,459.89","47,047.10",0.00,20,"87,075.45",0.05,1,"118,801.99",0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33351,1191210893,CAN_ALB,2050,"1,042,433.46","33,692.00",0.00,20,"67,057.91",0.05,1,"89,028.46",0.00,0.00
33352,1191210894,CAN_ALB,2020,"1,393,459.89","47,047.10",0.00,20,"59,189.60",0.05,1,"116,564.35",0.00,0.00
33353,1191210894,CAN_ALB,2030,"1,247,479.88","38,866.50",0.00,20,"59,189.60",0.05,1,"104,850.54",0.00,321.39
33354,1191210894,CAN_ALB,2040,"1,120,977.80","35,882.70",0.00,20,"59,189.60",0.05,1,"94,699.69",0.00,0.00


In [118]:
BLD_YRS_FOR_GEN_PERIOD = pd.merge(left=generation_projects_info[['GENERATION_PROJECT', 'gen_load_zone', 'gen_max_age', 'scenario']], right=period_info[['investment_period', 'period_start', 'period_length_years','scenario']], on='scenario')
BLD_YRS_FOR_GEN_PERIOD = pd.merge(left=BLD_YRS_FOR_GEN_PERIOD, right=gen_build_costs[['GENERATION_PROJECT', 'build_year', 'scenario']], on=['GENERATION_PROJECT', 'scenario'])
BLD_YRS_FOR_GEN_PERIOD.loc[:,'operation']=BLD_YRS_FOR_GEN_PERIOD.apply(lambda x: gen_build_can_operate_in_period(x['gen_max_age'], x['build_year'], x['investment_period'], x['period_start'], x['period_length_years']), axis=1)    
BLD_YRS_FOR_GEN_PERIOD

,GENERATION_PROJECT,gen_load_zone,gen_max_age,scenario,investment_period,period_start,period_length_years,build_year,operation
0,77333,CA_SCE_CEN,20,1,2020,2016,10,2020,1
1,77333,CA_SCE_CEN,20,1,2020,2016,10,2030,0
2,77333,CA_SCE_CEN,20,1,2020,2016,10,2040,0
3,77333,CA_SCE_CEN,20,1,2020,2016,10,2050,0
4,77333,CA_SCE_CEN,20,1,2030,2026,10,2020,1
...,...,...,...,...,...,...,...,...,...
133419,1191210894,CAN_ALB,20,1,2040,2036,10,2050,0
133420,1191210894,CAN_ALB,20,1,2050,2046,10,2020,0
133421,1191210894,CAN_ALB,20,1,2050,2046,10,2030,0
133422,1191210894,CAN_ALB,20,1,2050,2046,10,2040,1


In [119]:
gen_costs=pd.merge(left=BLD_YRS_FOR_GEN_PERIOD[['scenario', 'GENERATION_PROJECT', 'gen_load_zone', 'investment_period', 'build_year', 'operation']], 
                   right=gen_build_costs_extended[['scenario', 'gen_load_zone', 'GENERATION_PROJECT','build_year','BuildGen', 'gen_capital_cost_annual', 'gen_fixed_om']]
                   , on=['GENERATION_PROJECT', 'gen_load_zone', 'scenario', 'build_year'])

gen_costs['GenCapitalCosts'] = gen_costs['BuildGen'] * gen_costs['gen_capital_cost_annual'] *gen_costs['operation']
gen_costs['GenFixedOMCosts'] = gen_costs['BuildGen'] * gen_costs['gen_fixed_om'] *gen_costs['operation']
gen_costs['TotalGenFixedCosts'] = gen_costs['GenCapitalCosts'] + gen_costs['GenFixedOMCosts']*gen_costs['operation']
gen_costs

,scenario,GENERATION_PROJECT,gen_load_zone,investment_period,build_year,operation,BuildGen,gen_capital_cost_annual,gen_fixed_om,GenCapitalCosts,GenFixedOMCosts,TotalGenFixedCosts
0,1,77333,CA_SCE_CEN,2020,2020,1,0.00,"118,803.08","47,047.10",0.00,0.00,0.00
1,1,77333,CA_SCE_CEN,2030,2020,1,0.00,"118,803.08","47,047.10",0.00,0.00,0.00
2,1,77333,CA_SCE_CEN,2040,2020,0,0.00,"118,803.08","47,047.10",0.00,0.00,0.00
3,1,77333,CA_SCE_CEN,2050,2020,0,0.00,"118,803.08","47,047.10",0.00,0.00,0.00
4,1,77333,CA_SCE_CEN,2020,2030,0,0.00,"107,089.27","38,866.50",0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
133419,1,1191210894,CAN_ALB,2050,2040,1,0.00,"94,699.69","35,882.70",0.00,0.00,0.00
133420,1,1191210894,CAN_ALB,2020,2050,0,321.39,"88,397.08","33,692.00",0.00,0.00,0.00
133421,1,1191210894,CAN_ALB,2030,2050,0,321.39,"88,397.08","33,692.00",0.00,0.00,0.00
133422,1,1191210894,CAN_ALB,2040,2050,0,321.39,"88,397.08","33,692.00",0.00,0.00,0.00


In [120]:
if len(analysis_zones)>0:
   gen_costs = gen_costs.loc[gen_costs.gen_load_zone.isin(analysis_zones)]

In [121]:
TotalGenFixedCosts = gen_costs.pivot_table(
    index=['scenario','investment_period'], values="TotalGenFixedCosts", aggfunc=np.sum )
TotalGenFixedCosts.rename(columns={'TotalGenFixedCosts' : 'AnnualCost_Real'}, inplace=True)
TotalGenFixedCosts.reset_index(inplace=True)
TotalGenFixedCosts = pd.merge(TotalGenFixedCosts, period_info[['scenario', 'investment_period', 'bring_annual_costs_to_base_year']], on=['scenario', 'investment_period'])
TotalGenFixedCosts ['AnnualCost_NPV'] = TotalGenFixedCosts ['AnnualCost_Real'] * TotalGenFixedCosts ['bring_annual_costs_to_base_year']
TotalGenFixedCosts.drop(['bring_annual_costs_to_base_year'], axis=1, inplace=True)
TotalGenFixedCosts['Component'] = 'TotalGenFixedCosts'
TotalGenFixedCosts['Component_type'] = 'annual'
TotalGenFixedCosts = TotalGenFixedCosts[['scenario', 'investment_period', 'Component', 'Component_type', 'AnnualCost_NPV', 'AnnualCost_Real']]
TotalGenFixedCosts

,scenario,investment_period,Component,Component_type,AnnualCost_NPV,AnnualCost_Real
0,1,2020,TotalGenFixedCosts,annual,"48,715,761,181.18","5,722,370,925.97"
1,1,2030,TotalGenFixedCosts,annual,"81,729,037,825.29","15,637,808,466.36"
2,1,2040,TotalGenFixedCosts,annual,"110,492,260,182.77","34,436,923,951.48"
3,1,2050,TotalGenFixedCosts,annual,"122,116,328,423.34","61,995,365,254.59"


### Storage Energy Fixed Costs

In [122]:
storage_build_costs_extended = gen_build_costs_extended.copy()

BuildStorageEnergy = get_data(scenario, "BuildStorageEnergy.csv")
BuildStorageEnergy  = BuildStorageEnergy .replace({"scenario": short_names})
BuildStorageEnergy.rename(columns={'STORAGE_GEN_BLD_YRS_1':'GENERATION_PROJECT', "STORAGE_GEN_BLD_YRS_2": "build_year"},inplace=True)

storage_build_costs_extended = pd.merge(left=storage_build_costs_extended, right=BuildStorageEnergy, on=['GENERATION_PROJECT', 'build_year', 'scenario'])
storage_build_costs_extended

,GENERATION_PROJECT,gen_load_zone,build_year,gen_overnight_cost,gen_fixed_om,gen_storage_energy_overnight_cost,gen_max_age,gen_connect_cost_per_mw,interest_rate,scenario,gen_capital_cost_annual,storage_energy_capital_cost_annual,BuildGen,BuildStorageEnergy
0,154496,CA_IID,2016,0.00,0.00,0.00,10,0.00,0.05,1,0.00,0.00,30.00,20.00
1,154496,CA_IID,2020,0.00,0.00,0.00,10,0.00,0.05,1,0.00,0.00,0.00,68.20
2,154886,CA_SCE_S,2012,0.00,0.00,0.00,10,0.00,0.05,1,0.00,0.00,1.00,6.80
3,155173,CA_SCE_S,2016,0.00,0.00,0.00,10,0.00,0.05,1,0.00,0.00,20.00,20.00
4,155173,CA_SCE_S,2020,0.00,0.00,0.00,10,0.00,0.05,1,0.00,0.00,0.00,38.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,1191209768,WY_SE,2050,"113,216.20","15,834.00","130,034.60",10,"72,469.68",0.05,1,"24,047.17","16,840.08",0.00,0.00
306,1191209769,WY_SW,2020,"414,708.30","32,043.49","295,794.67",10,"70,251.14",0.05,1,"62,804.47","38,306.76",0.00,0.00
307,1191209769,WY_SW,2030,"150,026.30","20,981.90","172,312.80",10,"70,251.14",0.05,1,"28,526.94","22,315.30",0.00,0.00
308,1191209769,WY_SW,2040,"126,912.00","17,749.30","145,764.70",10,"70,251.14",0.05,1,"25,533.53","18,877.20",0.00,0.00


In [123]:
storage_costs=pd.merge(left=BLD_YRS_FOR_GEN_PERIOD[['scenario', 'GENERATION_PROJECT', 'gen_load_zone', 'investment_period', 'build_year', 'operation']], 
                   right=storage_build_costs_extended[['scenario', 'GENERATION_PROJECT', 'gen_load_zone', 'build_year','BuildStorageEnergy', 'storage_energy_capital_cost_annual']]
                   , on=['GENERATION_PROJECT', 'gen_load_zone', 'scenario', 'build_year'])

storage_costs['StorageEnergyFixedCost'] = storage_costs['BuildStorageEnergy'] * storage_costs['storage_energy_capital_cost_annual'] *storage_costs['operation']
storage_costs

,scenario,GENERATION_PROJECT,gen_load_zone,investment_period,build_year,operation,BuildStorageEnergy,storage_energy_capital_cost_annual,StorageEnergyFixedCost
0,1,154496,CA_IID,2020,2016,1,20.00,0.00,0.00
1,1,154496,CA_IID,2030,2016,0,20.00,0.00,0.00
2,1,154496,CA_IID,2040,2016,0,20.00,0.00,0.00
3,1,154496,CA_IID,2050,2016,0,20.00,0.00,0.00
4,1,154496,CA_IID,2020,2020,1,68.20,0.00,0.00
...,...,...,...,...,...,...,...,...,...
1235,1,1191209769,WY_SW,2050,2040,0,0.00,"18,877.20",0.00
1236,1,1191209769,WY_SW,2020,2050,0,0.00,"16,840.08",0.00
1237,1,1191209769,WY_SW,2030,2050,0,0.00,"16,840.08",0.00
1238,1,1191209769,WY_SW,2040,2050,0,0.00,"16,840.08",0.00


In [124]:
if len(analysis_zones)>0:
   storage_costs = storage_costs.loc[storage_costs.gen_load_zone.isin(analysis_zones)]

In [125]:
StorageEnergyFixedCost = storage_costs.pivot_table(
    index=['scenario','investment_period'], values="StorageEnergyFixedCost", aggfunc=np.sum )
StorageEnergyFixedCost.rename(columns={'StorageEnergyFixedCost' : 'AnnualCost_Real'}, inplace=True)
StorageEnergyFixedCost.reset_index(inplace=True)
StorageEnergyFixedCost = pd.merge(StorageEnergyFixedCost, period_info[['scenario', 'investment_period', 'bring_annual_costs_to_base_year']], on=['scenario', 'investment_period'])
StorageEnergyFixedCost ['AnnualCost_NPV'] = StorageEnergyFixedCost ['AnnualCost_Real'] * StorageEnergyFixedCost ['bring_annual_costs_to_base_year']
StorageEnergyFixedCost.drop(['bring_annual_costs_to_base_year'], axis=1, inplace=True)
StorageEnergyFixedCost['Component'] = 'StorageEnergyFixedCost'
StorageEnergyFixedCost['Component_type'] = 'annual'
StorageEnergyFixedCost = StorageEnergyFixedCost[['scenario', 'investment_period', 'Component', 'Component_type', 'AnnualCost_NPV', 'AnnualCost_Real']]
StorageEnergyFixedCost

,scenario,investment_period,Component,Component_type,AnnualCost_NPV,AnnualCost_Real
0,1,2020,StorageEnergyFixedCost,annual,0.00,0.00
1,1,2030,StorageEnergyFixedCost,annual,0.00,0.00
2,1,2040,StorageEnergyFixedCost,annual,"10,046,917,307.41","3,131,304,642.42"
3,1,2050,StorageEnergyFixedCost,annual,"33,125,074,483.66","16,816,760,855.93"


### Generation Variable Costs

In [126]:
dispatch = get_data(scenario, "dispatch.csv")
dispatch = dispatch[['generation_project', 'timestamp', 'gen_load_zone', 'period','VariableOMCost_per_yr', 'scenario']]
dispatch  = dispatch  .replace({"scenario": short_names})
dispatch.rename(columns={'period': 'investment_period', 'VariableOMCost_per_yr': 'GenVariableOMCosts'}, inplace=True)
dispatch

exception
timestamp in column


,generation_project,timestamp,gen_load_zone,investment_period,GenVariableOMCosts,scenario
0,77333,2020011602,CA_SCE_CEN,2020,0.00,1
1,77333,2020011606,CA_SCE_CEN,2020,0.00,1
2,77333,2020011610,CA_SCE_CEN,2020,0.00,1
3,77333,2020011614,CA_SCE_CEN,2020,0.00,1
4,77333,2020011618,CA_SCE_CEN,2020,0.00,1
...,...,...,...,...,...,...
5131867,1191210894,2050122107,CAN_ALB,2050,0.00,1
5131868,1191210894,2050122111,CAN_ALB,2050,0.00,1
5131869,1191210894,2050122115,CAN_ALB,2050,0.00,1
5131870,1191210894,2050122119,CAN_ALB,2050,0.00,1


In [127]:
if len(analysis_zones)>0:
   dispatch = dispatch.loc[dispatch.gen_load_zone.isin(analysis_zones)]

In [128]:
GenVariableOMCostsInTP = dispatch.pivot_table(
    index=['scenario','investment_period'], values="GenVariableOMCosts", aggfunc=np.sum )
GenVariableOMCostsInTP.rename(columns={'GenVariableOMCosts' : 'AnnualCost_Real'}, inplace=True)
GenVariableOMCostsInTP.reset_index(inplace=True)
GenVariableOMCostsInTP = pd.merge(GenVariableOMCostsInTP, period_info[['scenario', 'investment_period', 'bring_annual_costs_to_base_year']], on=['scenario', 'investment_period'])
GenVariableOMCostsInTP ['AnnualCost_NPV'] = GenVariableOMCostsInTP ['AnnualCost_Real'] * GenVariableOMCostsInTP ['bring_annual_costs_to_base_year']
GenVariableOMCostsInTP.drop(['bring_annual_costs_to_base_year'], axis=1, inplace=True)
GenVariableOMCostsInTP['Component'] = 'GenVariableOMCostsInTP'
GenVariableOMCostsInTP['Component_type'] = 'timepoint'
GenVariableOMCostsInTP = GenVariableOMCostsInTP[['scenario', 'investment_period', 'Component', 'Component_type', 'AnnualCost_NPV', 'AnnualCost_Real']]
GenVariableOMCostsInTP

,scenario,investment_period,Component,Component_type,AnnualCost_NPV,AnnualCost_Real
0,1,2020,GenVariableOMCostsInTP,timepoint,"16,871,991,468.17","1,981,859,486.54"
1,1,2030,GenVariableOMCostsInTP,timepoint,"9,115,294,013.00","1,744,095,191.66"
2,1,2040,GenVariableOMCostsInTP,timepoint,"3,592,360,023.82","1,119,624,385.83"
3,1,2050,GenVariableOMCostsInTP,timepoint,"400,168,689.33","203,155,502.45"


### Transmission Fixed Costs

In [129]:
transmission = get_data(scenario, "transmission.csv")
transmission = transmission[['PERIOD', 'trans_lz1', 'trans_lz2','TotalAnnualCost', 'scenario']]
transmission  = transmission  .replace({"scenario": short_names})
transmission.rename(columns={'TotalAnnualCost': 'TxFixedCosts', 'PERIOD':'investment_period'}, inplace=True)
transmission


,investment_period,trans_lz1,trans_lz2,TxFixedCosts,scenario
0,2020,AZ_APS_E,AZ_APS_N,0.00,1
1,2030,AZ_APS_E,AZ_APS_N,0.00,1
2,2040,AZ_APS_E,AZ_APS_N,0.00,1
3,2050,AZ_APS_E,AZ_APS_N,0.00,1
4,2020,AZ_APS_E,AZ_NM_N,0.00,1
...,...,...,...,...,...
499,2050,WY_NW,WY_SW,0.00,1
500,2020,WY_SE,WY_SW,0.00,1
501,2030,WY_SE,WY_SW,0.00,1
502,2040,WY_SE,WY_SW,"378,070,000.00",1


In [130]:
if len(analysis_zones)>0:
   transmission  = transmission.loc[(transmission.trans_lz1.isin(analysis_zones)) | (transmission.trans_lz2.isin(analysis_zones))]

In [131]:
TxFixedCosts = transmission.pivot_table(
    index=['scenario','investment_period'], values="TxFixedCosts", aggfunc=np.sum )
TxFixedCosts.rename(columns={'TxFixedCosts' : 'AnnualCost_Real'}, inplace=True)
TxFixedCosts.reset_index(inplace=True)
TxFixedCosts = pd.merge(TxFixedCosts, period_info[['scenario', 'investment_period', 'bring_annual_costs_to_base_year']], on=['scenario', 'investment_period'])
TxFixedCosts ['AnnualCost_NPV'] = TxFixedCosts ['AnnualCost_Real'] * TxFixedCosts ['bring_annual_costs_to_base_year']
TxFixedCosts.drop(['bring_annual_costs_to_base_year'], axis=1, inplace=True)
TxFixedCosts['Component'] = 'TxFixedCosts'
TxFixedCosts['Component_type'] = 'annual'
TxFixedCosts = TxFixedCosts[['scenario', 'investment_period', 'Component', 'Component_type', 'AnnualCost_NPV', 'AnnualCost_Real']]
TxFixedCosts

,scenario,investment_period,Component,Component_type,AnnualCost_NPV,AnnualCost_Real
0,1,2020,TxFixedCosts,annual,0.00,0.00
1,1,2030,TxFixedCosts,annual,"489,102,907.27","93,583,600.00"
2,1,2040,TxFixedCosts,annual,"7,317,534,137.12","2,280,642,700.00"
3,1,2050,TxFixedCosts,annual,"17,351,963,237.02","8,809,152,000.00"


### Fuel Costs

In [132]:
fname = "zone_to_regional_fuel_market.csv"
zone_to_rfm = get_data(scenario,fname, fpath='inputs')
zone_to_rfm  = zone_to_rfm .replace({"scenario": short_names})
zone_to_rfm 

,load_zone,regional_fuel_market,scenario
0,AZ_APS_E,AZ_APS_E-Bio_Solid,1
1,AZ_APS_N,AZ_APS_N-Bio_Solid,1
2,AZ_APS_SW,AZ_APS_SW-Bio_Solid,1
3,AZ_NM_N,AZ_NM_N-Bio_Solid,1
4,AZ_NW,AZ_NW-Bio_Solid,1
...,...,...,...
95,WA_W,WA_W-Bio_Gas,1
96,WY_NE,WY_NE-Bio_Gas,1
97,WY_NW,WY_NW-Bio_Gas,1
98,WY_SE,WY_SE-Bio_Gas,1


In [133]:
fuel_costs_tier_0 = get_data(scenario,"fuel_cost.csv", fpath='inputs')
fuel_costs_tier_0['regional_fuel_market'] = fuel_costs_tier_0['load_zone'] + '_' + fuel_costs_tier_0['fuel']
fuel_costs_tier_0['tier'] = 0
fuel_costs_tier_0.rename(columns={'fuel_cost':'unit_cost'},inplace=True)
fuel_costs_tier_0 = fuel_costs_tier_0[['regional_fuel_market','period','tier','unit_cost','scenario','load_zone']]
fuel_costs_tier_0 = fuel_costs_tier_0 .replace({"scenario": short_names})
fuel_costs_tier_0

,regional_fuel_market,period,tier,unit_cost,scenario,load_zone
0,AZ_APS_E_Bio_Liquid,2020,0,0.01,1,AZ_APS_E
1,AZ_APS_E_Bio_Liquid,2030,0,0.01,1,AZ_APS_E
2,AZ_APS_E_Bio_Liquid,2040,0,0.01,1,AZ_APS_E
3,AZ_APS_E_Bio_Liquid,2050,0,0.01,1,AZ_APS_E
4,AZ_APS_E_Coal,2020,0,1.97,1,AZ_APS_E
...,...,...,...,...,...,...
1195,WY_SW_ResidualFuelOil,2050,0,19.00,1,WY_SW
1196,WY_SW_Uranium,2020,0,0.68,1,WY_SW
1197,WY_SW_Uranium,2030,0,0.88,1,WY_SW
1198,WY_SW_Uranium,2040,0,1.15,1,WY_SW


In [134]:
fuel_costs_tiers_1_and_up = get_data(scenario,"fuel_supply_curves.csv", fpath='inputs')
fuel_costs_tiers_1_and_up = fuel_costs_tiers_1_and_up .replace({"scenario": short_names})
fuel_costs_tiers_1_and_up = fuel_costs_tiers_1_and_up[ ['regional_fuel_market', 'period', 'tier', 'unit_cost', 'scenario']]
fuel_costs_tiers_1_and_up = pd.merge(left=fuel_costs_tiers_1_and_up,right=zone_to_rfm , on=['regional_fuel_market', 'scenario'])
fuel_costs_tiers_1_and_up

,regional_fuel_market,period,tier,unit_cost,scenario,load_zone
0,AZ_APS_E-Bio_Solid,2020,1,1.93,1,AZ_APS_E
1,AZ_APS_E-Bio_Solid,2020,2,4.00,1,AZ_APS_E
2,AZ_APS_E-Bio_Solid,2020,3,487.51,1,AZ_APS_E
3,AZ_APS_E-Bio_Solid,2020,4,563.74,1,AZ_APS_E
4,AZ_APS_E-Bio_Solid,2020,5,637.87,1,AZ_APS_E
...,...,...,...,...,...,...
2086,WY_SE-Bio_Gas,2050,1,0.00,1,WY_SE
2087,WY_SW-Bio_Gas,2020,1,0.00,1,WY_SW
2088,WY_SW-Bio_Gas,2030,1,0.00,1,WY_SW
2089,WY_SW-Bio_Gas,2040,1,0.00,1,WY_SW


In [135]:
fuel_costs = pd.concat([fuel_costs_tier_0,fuel_costs_tiers_1_and_up],ignore_index=True)
fuel_costs = fuel_costs .replace({"scenario": short_names})
fuel_costs

,regional_fuel_market,period,tier,unit_cost,scenario,load_zone
0,AZ_APS_E_Bio_Liquid,2020,0,0.01,1,AZ_APS_E
1,AZ_APS_E_Bio_Liquid,2030,0,0.01,1,AZ_APS_E
2,AZ_APS_E_Bio_Liquid,2040,0,0.01,1,AZ_APS_E
3,AZ_APS_E_Bio_Liquid,2050,0,0.01,1,AZ_APS_E
4,AZ_APS_E_Coal,2020,0,1.97,1,AZ_APS_E
...,...,...,...,...,...,...
3286,WY_SE-Bio_Gas,2050,1,0.00,1,WY_SE
3287,WY_SW-Bio_Gas,2020,1,0.00,1,WY_SW
3288,WY_SW-Bio_Gas,2030,1,0.00,1,WY_SW
3289,WY_SW-Bio_Gas,2040,1,0.00,1,WY_SW


In [136]:
consumefuel = get_data(scenario, "ConsumeFuelTier.csv")
consumefuel  = consumefuel .replace({"scenario": short_names})
consumefuel.rename(columns={'RFM_SUPPLY_TIERS_1': 'regional_fuel_market', 'RFM_SUPPLY_TIERS_2': 'period', 'RFM_SUPPLY_TIERS_3':'tier'}, inplace='True')
consumefuel = pd.merge(left=consumefuel,right=fuel_costs , on=['regional_fuel_market', 'period', 'tier' ,'scenario'])
consumefuel ['FuelCostsPerPeriod'] = consumefuel ['ConsumeFuelTier'] * consumefuel ['unit_cost']
consumefuel .rename(columns={'period':'investment_period'}, inplace=True)
consumefuel 

,regional_fuel_market,investment_period,tier,ConsumeFuelTier,scenario,unit_cost,load_zone,FuelCostsPerPeriod
0,AZ_APS_E-Bio_Gas,2020,1,"250,630.00",1,0.00,AZ_APS_E,0.00
1,AZ_APS_E-Bio_Gas,2030,1,"250,630.00",1,0.00,AZ_APS_E,0.00
2,AZ_APS_E-Bio_Gas,2040,1,"250,630.00",1,0.00,AZ_APS_E,0.00
3,AZ_APS_E-Bio_Gas,2050,1,"250,630.00",1,0.00,AZ_APS_E,0.00
4,AZ_APS_E-Bio_Solid,2020,1,"651,930.00",1,1.93,AZ_APS_E,"1,258,977.72"
...,...,...,...,...,...,...,...,...
3286,WY_SW_ResidualFuelOil,2050,0,0.00,1,19.00,WY_SW,0.00
3287,WY_SW_Uranium,2020,0,0.00,1,0.68,WY_SW,0.00
3288,WY_SW_Uranium,2030,0,0.00,1,0.88,WY_SW,0.00
3289,WY_SW_Uranium,2040,0,0.00,1,1.15,WY_SW,0.00


In [137]:
if len(analysis_zones)>0:
   consumefuel   = consumefuel.loc[consumefuel.load_zone.isin(analysis_zones)]

In [138]:
FuelCostsPerPeriod = consumefuel.pivot_table(
    index=['scenario','investment_period'], values="FuelCostsPerPeriod", aggfunc=np.sum )
FuelCostsPerPeriod.rename(columns={'FuelCostsPerPeriod' : 'AnnualCost_Real'}, inplace=True)
FuelCostsPerPeriod.reset_index(inplace=True)
FuelCostsPerPeriod = pd.merge(FuelCostsPerPeriod, period_info[['scenario', 'investment_period', 'bring_annual_costs_to_base_year']], on=['scenario', 'investment_period'])
FuelCostsPerPeriod ['AnnualCost_NPV'] = FuelCostsPerPeriod ['AnnualCost_Real'] * FuelCostsPerPeriod ['bring_annual_costs_to_base_year']
FuelCostsPerPeriod.drop(['bring_annual_costs_to_base_year'], axis=1, inplace=True)
FuelCostsPerPeriod['Component'] = 'FuelCostsPerPeriod'
FuelCostsPerPeriod['Component_type'] = 'annual'
FuelCostsPerPeriod = FuelCostsPerPeriod[['scenario', 'investment_period', 'Component', 'Component_type', 'AnnualCost_NPV', 'AnnualCost_Real']]
FuelCostsPerPeriod


,scenario,investment_period,Component,Component_type,AnnualCost_NPV,AnnualCost_Real
0,1,2020,FuelCostsPerPeriod,annual,"105,418,059,643.59","12,382,876,197.57"
1,1,2030,FuelCostsPerPeriod,annual,"72,435,544,995.70","13,859,617,205.10"
2,1,2040,FuelCostsPerPeriod,annual,"26,829,569,650.97","8,361,923,705.73"
3,1,2050,FuelCostsPerPeriod,annual,"1,533,941,864.03","778,743,411.04"


### Total Costs

In [147]:
costs_itemized = pd.concat([TotalGenFixedCosts,StorageEnergyFixedCost, FuelCostsPerPeriod, TxFixedCosts, GenVariableOMCostsInTP])
#costs_itemized.sort_values(by=['investment_period'], inplace=True)
costs_itemized['sc_order'] = costs_itemized.scenario.map(order)
costs_itemized=costs_itemized.sort_values(by=['sc_order', 'investment_period']).drop('sc_order',axis=1)
costs_itemized.reset_index(inplace=True,drop=True)
costs_itemized

,scenario,investment_period,Component,Component_type,AnnualCost_NPV,AnnualCost_Real
0,1,2020,TotalGenFixedCosts,annual,"48,715,761,181.18","5,722,370,925.97"
1,1,2020,StorageEnergyFixedCost,annual,0.00,0.00
2,1,2020,FuelCostsPerPeriod,annual,"105,418,059,643.59","12,382,876,197.57"
3,1,2020,TxFixedCosts,annual,0.00,0.00
4,1,2020,GenVariableOMCostsInTP,timepoint,"16,871,991,468.17","1,981,859,486.54"
5,1,2030,TotalGenFixedCosts,annual,"81,729,037,825.29","15,637,808,466.36"
6,1,2030,StorageEnergyFixedCost,annual,0.00,0.00
7,1,2030,FuelCostsPerPeriod,annual,"72,435,544,995.70","13,859,617,205.10"
8,1,2030,TxFixedCosts,annual,"489,102,907.27","93,583,600.00"
9,1,2030,GenVariableOMCostsInTP,timepoint,"9,115,294,013.00","1,744,095,191.66"


### Electricity costs

In [140]:
loads = get_data(scenario, "loads.csv", fpath='inputs')
loads.columns= loads.columns.str.lower()
loads  = loads.replace({"scenario": short_names})

loads = pd.merge(right=loads, left=time_info[['timepoint', 'scenario', 'tp_weight', 'ts_period']], on=['scenario', 'timepoint'])
loads.rename(columns={'ts_period' : 'investment_period'}, inplace=True)

loads = pd.merge(right=loads, left=period_info[['investment_period', 'period_length_years', 'scenario']], on=['scenario', 'investment_period'])

if len(analysis_zones)>0:
   loads  = loads.loc[loads.load_zone.isin(analysis_zones)]

loads['SystemDemand_MWh_in_period']=loads['tp_weight'] * loads['zone_demand_mw']
loads['SystemDemand_MWh_in_year']=loads['SystemDemand_MWh_in_period'] / loads['period_length_years']

loads_in_period = loads.pivot_table(index=['scenario', 'investment_period'], values=['SystemDemand_MWh_in_period', 'SystemDemand_MWh_in_year'], aggfunc=np.sum)
loads_in_period.reset_index(inplace=True)
loads_in_period

,scenario,investment_period,SystemDemand_MWh_in_period,SystemDemand_MWh_in_year
0,1,2020,"9,436,195,205.62","943,619,520.56"
1,1,2030,"10,646,939,453.35","1,064,693,945.34"
2,1,2040,"12,453,152,233.60","1,245,315,223.36"
3,1,2050,"14,789,879,742.89","1,478,987,974.29"


In [141]:
electricity_costs = costs_itemized.pivot_table(index=['scenario', 'investment_period'], values=['AnnualCost_NPV', 'AnnualCost_Real'], aggfunc=np.sum)
electricity_costs.reset_index(inplace=True)


electricity_costs = pd.merge(left=electricity_costs, right=loads_in_period, on=['scenario', 'investment_period'])
electricity_costs['EnergyCostNPV_per_MWh']=electricity_costs['AnnualCost_NPV']/electricity_costs['SystemDemand_MWh_in_period']
electricity_costs['EnergyCostReal_per_MWh']=electricity_costs['AnnualCost_Real']/electricity_costs['SystemDemand_MWh_in_year']
electricity_costs

,scenario,investment_period,AnnualCost_NPV,AnnualCost_Real,SystemDemand_MWh_in_period,SystemDemand_MWh_in_year,EnergyCostNPV_per_MWh,EnergyCostReal_per_MWh
0,1,2020,"171,005,812,292.94","20,087,106,610.09","9,436,195,205.62","943,619,520.56",18.12,21.29
1,1,2030,"163,768,979,741.26","31,335,104,463.12","10,646,939,453.35","1,064,693,945.34",15.38,29.43
2,1,2040,"158,278,641,302.08","49,330,419,385.46","12,453,152,233.60","1,245,315,223.36",12.71,39.61
3,1,2050,"174,527,476,697.37","88,603,177,024.01","14,789,879,742.89","1,478,987,974.29",11.80,59.91
